In [1]:
import pandas as pd
# from prophet import Prophet  # optional, keep commented if not installed
# Example: prepare a `ds` and `y` dataframe for Prophet or ARIMA inputs

In [2]:
# Forecasting (simple demo)
import pandas as pd
from pathlib import Path
import numpy as np

CLEANED_DIR = Path('d:/slooze_challenge/Slooze_Analysis/data/cleaned')
OUT_DIR = Path('d:/slooze_challenge/Slooze_Analysis/results')
OUT_DIR.mkdir(parents=True, exist_ok=True)

sales = pd.read_csv(CLEANED_DIR / 'sales_cleaned.csv', parse_dates=['date'], low_memory=False)

# Pick the top-selling SKU for a short demo forecast
top_sku = sales.groupby('sku').agg(total_qty=('quantity','sum')).reset_index().sort_values('total_qty', ascending=False).iloc[0]['sku']
print('Using top SKU for demo:', top_sku)

sku_ts = sales[sales['sku'] == top_sku].set_index('date').resample('D').sum()['quantity'].fillna(0)
monthly = sku_ts.resample('M').sum()

# Simple forecast: average of last 3 months -> next 6 months
last_avg = monthly[-3:].mean()
future_index = pd.date_range(monthly.index[-1] + pd.offsets.MonthBegin(1), periods=6, freq='M')
forecast = pd.Series([last_avg] * len(future_index), index=future_index)
fc_df = pd.DataFrame({'date': forecast.index, 'forecast_qty': forecast.values})
fc_df.to_csv(OUT_DIR / f'forecast_{str(top_sku)}.csv', index=False)
print('Saved simple forecast for', top_sku, 'to results folder')

# Optional: if pmdarima is installed, create an ARIMA forecast as comparison
try:
    from pmdarima import auto_arima
    model = auto_arima(monthly, seasonal=True, m=12, suppress_warnings=True)
    n_periods = 6
    fc = model.predict(n_periods=n_periods)
    arima_idx = pd.date_range(monthly.index[-1] + pd.offsets.MonthBegin(1), periods=n_periods, freq='M')
    pd.DataFrame({'date': arima_idx, 'arima_fc': fc}).to_csv(OUT_DIR / f'forecast_arima_{str(top_sku)}.csv', index=False)
    print('Saved ARIMA forecast (pmdarima)')
except Exception:
    print('pmdarima not available — simple forecast saved instead')


Using top SKU for demo: 1_HARDERSFIELD_8111
Saved simple forecast for 1_HARDERSFIELD_8111 to results folder
pmdarima not available — simple forecast saved instead


C:\Users\20207\AppData\Local\Temp\ipykernel_22160\880785160.py:17: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = sku_ts.resample('M').sum()
C:\Users\20207\AppData\Local\Temp\ipykernel_22160\880785160.py:21: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  future_index = pd.date_range(monthly.index[-1] + pd.offsets.MonthBegin(1), periods=6, freq='M')
